# Transfer learning - Cats vs. Dogs

Code to initiliaze Tensorflow 2.0 in Colab

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt
import os

**[TODO] Create a data loader function that returns a tuple with a tf.float32 tensor for the image and a label. Images must be resized to 128x128.**
**N.B.: filenames are formatted as class.number.jpg**

In [132]:
def load_image(img_path, resize_to=None):
    # read the image from disk, decode it, convert the data type to
    # floating point, and resize it
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    if resize_to is not None:
        image = tf.image.resize(image, resize_to)
    # parse the class label from the file path
    fn_img = tf.strings.split(img_path, sep='/')[-1]
    label = tf.strings.split(fn_img, sep='.')[0]
    label = tf.strings.to_number(label, tf.float32)

    # return the image and the label
    return image, label

def load_dataset(folder):
    features_list = []
    labels_list = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            filepath = os.path.join(folder, filename)
            features, label = load_image(filepath, resize_to=(128, 128))
            features_list.append(features)
            labels_list.append(label)
    ds_features = tf.data.Dataset.from_tensor_slices(features_list)
    ds_labels = tf.data.Dataset.from_tensor_slices(labels_list)
    return tf.data.Dataset.zip((ds_features, ds_labels))

**[TODO] Create a tf.Dataset, map the loader function and prepare a batch object for training**

In [98]:
# Datasets creation
# 0 -> cats; 1 -> dogs
ds_train = load_dataset('./train/')
ds_test = load_dataset('./test/')

In [99]:
# Batch preparation
ds_train = ds_train.batch(batch_size=32)
# ds_train = ds_train.shuffle(128)
ds_test = ds_test.batch(batch_size=32)
# ds_test = ds_test.shuffle(128)

**Prepare Keras callback for Tensorboard**

In [100]:
logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, update_freq='batch')

**[TODO] Import the MobileNetV2 model trained on ImageNet without the final layer**

In [101]:
mnv2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(128, 128, 3), include_top=False, classes=2)
mnv2.trainable = False

**[TODO] Add a final classification layer for 2 classes and create the final Keras model**

In [109]:
inputs = tf.keras.Input(shape=(128, 128, 3))
h = mnv2(inputs, training=False)
# h = tf.keras.layers.GlobalAveragePooling2D()(h)
h = tf.keras.layers.Flatten()(h)
# h = tf.keras.layers.Dropout(0.2)(h)
classifier = tf.keras.layers.Dense(2, activation='softmax')(h)

**[TODO] Compile the Keras model: specify the optimization algorithm, the loss function and the test metric**

In [110]:
model = tf.keras.Model(inputs=inputs, outputs=classifier)
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**[TODO] Train the Keras model**

In [111]:
model.fit(ds_train, epochs=6)

Epoch 1/6
63/63 [==============================] - 12s 133ms/step - loss: 60.3277 - accuracy: 0.8695
Epoch 2/6
63/63 [==============================] - 9s 144ms/step - loss: 218.8498 - accuracy: 0.7080
Epoch 3/6
63/63 [==============================] - 9s 146ms/step - loss: 32.1931 - accuracy: 0.9115
Epoch 4/6
63/63 [==============================] - 9s 147ms/step - loss: 10.3578 - accuracy: 0.9560
Epoch 5/6
63/63 [==============================] - 9s 143ms/step - loss: 6.1719 - accuracy: 0.9675
Epoch 6/6
63/63 [==============================] - 9s 150ms/step - loss: 3.8354 - accuracy: 0.9740


**[TODO] Print model summary**

In [112]:
print(model.summary())

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 mobilenetv2_1.00_128 (Func  (None, 4, 4, 1280)        2257984   
 tional)                                                         
                                                                 
 flatten_10 (Flatten)        (None, 20480)             0         
                                                                 
 dense_13 (Dense)            (None, 2)                 40962     
                                                                 
Total params: 2298946 (8.77 MB)
Trainable params: 40962 (160.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
None


**[TODO] Test the Keras model by computing the accuracy the whole test set**

In [113]:
model.evaluate(ds_test)

32/32 [==============================] - 7s 147ms/step - loss: 10.0038 - accuracy: 0.9520


[10.003751754760742, 0.9520000219345093]

**[TODO] Load Test image 'test/0.1047.jpg', visualize it and check the network prediction**

In [136]:
# Showing the image
data, label = load_image('./test/0.1041.jpg', resize_to=(128, 128))
ds_img_data = tf.data.Dataset.from_tensor_slices([data])
plt.figure(figsize=(5, 5))
plt.imshow(ds_img_data.take(1)[0])
plt.axis('off')
plt.show()

TypeError: '_TakeDataset' object is not subscriptable

<Figure size 500x500 with 0 Axes>

In [115]:
# Predicting it
ds_img_data = tf.data.Dataset.from_tensor_slices([data])
model.predict(ds_img_data.batch(16))

1/1 [==============================] - 0s 36ms/step


array([[1., 0.]], dtype=float32)